# Resilient Distributed Datasets (복원 분산 데이터셋)
- Apache Spark의 기본적인 자료구조는 RDD 자료구조입니다.
- RDD는 스파크의 클러스터 내부의 노드들 간에 공유되어 병렬적으로 연산될 수 있는 자료구조입니다.
- 참고 : http://wallizard.com/eric/share/bigdata/ebook.2018/data-analytics-spark-using-python.pdf
4 Learning Spark Programming Basics

## Spark 설치

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz 
!tar -xvzf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

import os
import findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

findspark.init()
findspark.find()

spark-3.1.2-bin-hadoop2.7/
spark-3.1.2-bin-hadoop2.7/R/
spark-3.1.2-bin-hadoop2.7/R/lib/
spark-3.1.2-bin-hadoop2.7/R/lib/sparkr.zip
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/worker/
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/worker/worker.R
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/worker/daemon.R
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/tests/
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/tests/testthat/
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/tests/testthat/test_basic.R
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/profile/
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/profile/shell.R
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/profile/general.R
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/R/
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/R/SparkR
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/R/SparkR.rdx
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/R/SparkR.rdb
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/Meta/
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/Meta/features.rds
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/Meta/pac

'/content/spark-3.1.2-bin-hadoop2.7'

In [2]:
from pyspark import SparkContext, SparkConf
sc = SparkContext.getOrCreate()

## 파일에서 단어 검색

In [3]:
!ls /content/spark-3.1.2-bin-hadoop2.7/licenses/

LICENSE-AnchorJS.txt			       LICENSE-jsp-api.txt
LICENSE-antlr.txt			       LICENSE-JTransforms.txt
LICENSE-arpack.txt			       LICENSE-kryo.txt
LICENSE-automaton.txt			       LICENSE-leveldbjni.txt
LICENSE-bootstrap.txt			       LICENSE-machinist.txt
LICENSE-CC0.txt				       LICENSE-matchMedia-polyfill.txt
LICENSE-cloudpickle.txt			       LICENSE-minlog.txt
LICENSE-d3.min.js.txt			       LICENSE-modernizr.txt
LICENSE-dagre-d3.txt			       LICENSE-mustache.txt
LICENSE-datatables.txt			       LICENSE-netlib.txt
LICENSE-dnsjava.txt			       LICENSE-paranamer.txt
LICENSE-f2j.txt				       LICENSE-pmml-model.txt
LICENSE-graphlib-dot.txt		       LICENSE-protobuf.txt
LICENSE-istack-commons-runtime.txt	       LICENSE-py4j.txt
LICENSE-jakarta.activation-api.txt	       LICENSE-pyrolite.txt
LICENSE-jakarta-annotation-api		       LICENSE-re2j.txt
LICENSE-jakarta-ws-rs-api		       LICENSE-reflectasm.txt
LICENSE-jakarta.xml.bind-api.txt	       LICENSE-respond.txt
LICENSE-janino.txt			       LICEN

In [4]:
!cat /content/spark-3.1.2-bin-hadoop2.7/licenses/LICENSE-zstd.txt

BSD License

For Zstandard software

Copyright (c) 2016-present, Facebook, Inc. All rights reserved.

Redistribution and use in source and binary forms, with or without modification,
are permitted provided that the following conditions are met:

 * Redistributions of source code must retain the above copyright notice, this
   list of conditions and the following disclaimer.

 * Redistributions in binary form must reproduce the above copyright notice,
   this list of conditions and the following disclaimer in the documentation
   and/or other materials provided with the distribution.

 * Neither the name Facebook nor the names of its contributors may be used to
   endorse or promote products derived from this software without specific
   prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND
ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED
WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE A

In [5]:
# 로컬 파일시스템에서 파일 로드
rdd1 = sc.textFile("file:///content/spark-3.1.2-bin-hadoop2.7/licenses/LICENSE-*")

# 파일 레코드 필터링
rdd2 = rdd1.filter(lambda line: "SOFTWARE" in line)

# 파일로 저장
rdd2.saveAsTextFile("file:///tmp/output")

In [6]:
!cat  /tmp/output/part-00000

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND
SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.


## textFile() 메소드를 사용해 RDD 만들기

In [7]:
# licenses RDD에 전체 디렉토리의 내용 로드
license_files = sc.textFile("file:///content/spark-3.1.2-bin-hadoop2.7/licenses/")

In [8]:
# 생성된 객체 검사
license_files

file:///content/spark-3.1.2-bin-hadoop2.7/licenses/ MapPartitionsRDD[6] at textFile at NativeMethodAccessorImpl.java:0

In [9]:
license_files.take(1)

['JTransforms']

In [10]:
license_files.getNumPartitions()

57

In [11]:
# 모든 파일에서 총 라인 수
license_files.count()

2973

## 데이터 소스에서 RDD 생성

`$SPARK_HOME/bin/pyspark \
--driver-class-path mysql-connector-java-5.*-bin.jar \
--master local
Once you have launch`

In [12]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('rdd').getOrCreate()

`spark.read.jdbc(url, table,
    column=None,
    lowerBound=None,
    upperBound=None,
    numPartitions=None,
    predicates=None,
    properties=None)`

## JSON 파일에서 RDD 생성

In [13]:
people = spark.read.json("/content/spark-3.1.2-bin-hadoop2.7/examples/src/main/resources/people.json")

In [14]:
people

DataFrame[age: bigint, name: string]

In [15]:
people.dtypes

[('age', 'bigint'), ('name', 'string')]

In [16]:
people.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [17]:
from pyspark.sql import SQLContext

# as with all DataFrames you can create use them to run SQL queries as follows
sqlContext = SQLContext(sc)
sqlContext.registerDataFrameAsTable(people, "people")
df2 = spark.sql("SELECT name, age FROM people WHERE age > 20")
df2.show()

+----+---+
|name|age|
+----+---+
|Andy| 30|
+----+---+



## 프로그래밍 방식으로 RDD 생성
`parallelize()`

In [18]:
parallelrdd = sc.parallelize([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [19]:
parallelrdd

ParallelCollectionRDD[21] at readRDDFromFile at PythonRDD.scala:274

In [20]:
parallelrdd.count()

9

In [21]:
# 0에서 시작해서 1000개의 정수로, 2개의 파티션에서 1씩 증가하는 RDD 생성
range_rdd = sc.range(0, 1000, 1, 2)
range_rdd

PythonRDD[24] at RDD at PythonRDD.scala:53

In [22]:
range_rdd.getNumPartitions()

2

In [23]:
range_rdd.max()

999

## Saving and Loading SequenceFiles
- Similarly to text files, SequenceFiles can be saved and loaded by specifying the path.   
- The key and value classes can be specified, but for standard Writables this is not required.

In [24]:
rdd = sc.parallelize(range(1, 4)).map(lambda x: (x, "a" * x))
rdd.saveAsSequenceFile("./sf")
sorted(sc.sequenceFile("./sf").collect())

[(1, 'a'), (2, 'aa'), (3, 'aaa')]

In [25]:
range_rdd.take(5)

[0, 1, 2, 3, 4]

## RDD 연산

In [26]:
originalrdd = sc.parallelize([0, 1, 2, 3, 4, 5, 6, 7, 8])
newrdd = originalrdd.filter(lambda x: x % 2)

In [27]:
newrdd.collect()

[1, 3, 5, 7]

## 기본 RDD 변환

In [28]:
!cat /content/spark-3.1.2-bin-hadoop2.7/licenses/LICENSE-AnchorJS.txt

The MIT License (MIT)

Copyright (c) <year> <copyright holders>

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in
all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,


In [29]:
licenses = sc.textFile('file:///content/spark-3.1.2-bin-hadoop2.7/licenses/LICENSE-AnchorJS.txt')

### map(), flatMap(), filter()

In [30]:
words = licenses.flatMap(lambda x: x.split(' '))
words.take(5)

['The', 'MIT', 'License', '(MIT)', '']

In [31]:
lowercase = words.map(lambda x: x.lower())
lowercase.take(5)

['the', 'mit', 'license', '(mit)', '']

In [32]:
longwords = lowercase.filter(lambda x: len(x) > 12)
longwords.take(2)

['documentation', 'merchantability,']

### count()

In [33]:
words = licenses.flatMap(lambda x: x.split(' '))
words.count()

175

### distinct()

In [34]:
words = licenses.flatMap(lambda x: x.split(' '))
lowercase = words.map(lambda x: x.lower())
allwords = lowercase.count()
distinctwords = lowercase.distinct().count()
print("Total words: {}, Distinct Words:{}".format(allwords, distinctwords))

Total words: 175, Distinct Words:107


### collect()

In [35]:
words = licenses.flatMap(lambda x: x.split(' '))
words.collect()

['The',
 'MIT',
 'License',
 '(MIT)',
 '',
 'Copyright',
 '(c)',
 '<year>',
 '<copyright',
 'holders>',
 '',
 'Permission',
 'is',
 'hereby',
 'granted,',
 'free',
 'of',
 'charge,',
 'to',
 'any',
 'person',
 'obtaining',
 'a',
 'copy',
 'of',
 'this',
 'software',
 'and',
 'associated',
 'documentation',
 'files',
 '(the',
 '"Software"),',
 'to',
 'deal',
 'in',
 'the',
 'Software',
 'without',
 'restriction,',
 'including',
 'without',
 'limitation',
 'the',
 'rights',
 'to',
 'use,',
 'copy,',
 'modify,',
 'merge,',
 'publish,',
 'distribute,',
 'sublicense,',
 'and/or',
 'sell',
 'copies',
 'of',
 'the',
 'Software,',
 'and',
 'to',
 'permit',
 'persons',
 'to',
 'whom',
 'the',
 'Software',
 'is',
 'furnished',
 'to',
 'do',
 'so,',
 'subject',
 'to',
 'the',
 'following',
 'conditions:',
 '',
 'The',
 'above',
 'copyright',
 'notice',
 'and',
 'this',
 'permission',
 'notice',
 'shall',
 'be',
 'included',
 'in',
 'all',
 'copies',
 'or',
 'substantial',
 'portions',
 'of',
 'th

### take()

In [36]:
words = licenses.flatMap(lambda x: x.split(' '))
words.take(3)

['The', 'MIT', 'License']

## PairRDD 변환

In [37]:
kvpairs = sc.parallelize([('city','Hayward')
    ,('state','CA')
    ,('zip',94541)
    ,('country','USA')])

### keys()

In [38]:
kvpairs.keys().collect()

['city', 'state', 'zip', 'country']

### values()

In [39]:
kvpairs.values().collect()

['Hayward', 'CA', 94541, 'USA']

### keyBy()

In [40]:
# city, country, location_no
locations = sc.parallelize([('Hayward', 'USA', 1)
    ,('Baumholder','Germany', 2)
    ,('Alexandria','USA', 3)
    ,('Melbourne','Australia', 4)])
bylocno = locations.keyBy(lambda x: x[2])
bylocno.collect()

[(1, ('Hayward', 'USA', 1)),
 (2, ('Baumholder', 'Germany', 2)),
 (3, ('Alexandria', 'USA', 3)),
 (4, ('Melbourne', 'Australia', 4))]